# CoT Explainability for RAG

Dieses Notebook zeigt, wie der Explainability-Prompt mit Retrieval-Trace genutzt wird.

basierend auf: https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=11053836&tag=1

### Was es kann?
- LLM erhält Documents mit Relevance Score und Doc-ID
- LLM wird gepromptet in seiner Antwort die verwendeten Doc-IDs als Quelle anzugeben. Idealerweise an den richtigen Stellen
- Es wird nicht auf die Richtigkeit der engültigen Antwort geschaut, sondern wie gut wurden die Quellen verwendet
    - aka cosine similarity der Evidence mit der echten Quelle.
- Es wird geschaut ob die richtigen Quellen verwendet werden


In [1]:
import sys, json
from pathlib import Path

# Füge das Projektverzeichnis (mit `src/`) dem Python-Pfad hinzu, egal von wo das Notebook gestartet wird.
project_root = next((p for p in [Path.cwd()] + list(Path.cwd().parents) if (p / 'src').exists()), None)
if project_root is None:
    raise RuntimeError("\"src\"-Verzeichnis nicht gefunden. Bitte Notebook im Projekt laufen lassen.")
root_str = str(project_root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)


In [2]:
from src.modules.cot_explainable import CoTExplainable, ExplainableAnswer, _format_documents
from src.modules.rag_engine import RAGEngine
from src.modules.llm_client import LLMClient
from src.modules.data_loader_single_hop import BoolQDataLoader

### Real data example

In [3]:
_dataLoader = BoolQDataLoader()
documents = _dataLoader.setup()

In [4]:
ragEngine = RAGEngine(persist_dir="../data/vector_db_bool")
ragEngine.setup(documents=documents)

Loading existing vector store from ../data/vector_db_bool...
RagEngine ready.


In [5]:
client = LLMClient(provider="ollama", model_name="deepseek-r1:1.5b")
explain_client = LLMClient(provider="ollama", model_name="gemma3:1b", structured_output=ExplainableAnswer)

question = "what is age of majority?"

documents = ragEngine.retrieve_documents(question)
context = _format_documents(documents, from_query=question)
message = client._create_final_answer_prompt(question, context)

llm = client.get_llm()
final_answer_response = llm.invoke(message)
print(message)
final_answer = final_answer_response.content.strip()

print(final_answer)

explainer = CoTExplainable(llm_client=explain_client)
response = explainer.explain(query=question, answer=final_answer, documents=documents)

Connecting to local Ollama (deepseek-r1:1.5b)...

            You are a helpful assistant. Using all the provided context from multiple search hops, you must answer the original question.
            If the context is not sufficient, state that you cannot answer the question with the given information.

            Original Question: what is age of majority?

            Full Context from all search hops:
            ('<doc id="chunk-1-1" from_hop="1" search_query="what is age of majority?">\nThe age of majority is the threshold of adulthood as recognized or declared in law. It is the moment when minors cease to be considered such and assume legal control over their persons, actions, and decisions, thus terminating the control and legal responsibilities of their parents or guardian over them. Most countries set the age of majority at 18. The word majority here refers to having greater years and being of full age as opposed to minority, the state of being a minor. The law in a given jur

In [6]:
print(CoTExplainable.prettify(response))

evidence: The question 'what is age of majority?' refers to the legal threshold beyond which individuals are considered adults.
  support: chunk-1-1, chunk-1-2
  rationale: This is the core definition provided in the context.
    step 1: The question asks for the definition of 'age of majority'.
      quote: The question 'what is age of majority?' refers to the legal threshold beyond which individuals are considered adults.
    step 2: The provided text defines age of majority as the threshold of adulthood as recognized or declared in law.
      quote: In many countries, this threshold is set at 18 years old.
    step 3: The text further clarifies that the term 'age of majority' can vary depending on jurisdiction and application.
      quote: The term 'age of majority' can vary depending on the jurisdiction and application.
evidence: In many countries, this threshold is set at 18 years old.
  support: chunk-1-1, chunk-1-2
  rationale: The text explicitly states this as a common practic

In [8]:
print(documents[0])
print(documents[1])
print(documents[2])
print(documents[3])

page_content='The age of majority is the threshold of adulthood as recognized or declared in law. It is the moment when minors cease to be considered such and assume legal control over their persons, actions, and decisions, thus terminating the control and legal responsibilities of their parents or guardian over them. Most countries set the age of majority at 18. The word majority here refers to having greater years and being of full age as opposed to minority, the state of being a minor. The law in a given jurisdiction may not actually use the term ``age of majority''. The term typically refers to a collection of laws bestowing the status of adulthood. The age of majority does not necessarily correspond to the mental or physical maturity of an individual.' metadata={'id': 7431, 'question': 'is 18 the legal age in every country', 'source': 'boolq', 'answer': False, 'split': 'train'}
page_content='In law, a minor is a person under a certain age, usually the age of majority, which legall